## 1. TV, halftime shows, and the Big Game
<p>Whether or not you like football, the Super Bowl is a spectacle. There's a little something for everyone at your Super Bowl party. Drama in the form of blowouts, comebacks, and controversy for the sports fan. There are the ridiculously expensive ads, some hilarious, others gut-wrenching, thought-provoking, and weird. The half-time shows with the biggest musicians in the world, sometimes <a href="https://youtu.be/ZD1QrIe--_Y?t=14">riding giant mechanical tigers</a> or <a href="https://youtu.be/mjrdywp5nyE?t=62">leaping from the roof of the stadium</a>. It's a show, baby. And in this notebook, we're going to find out how some of the elements of this show interact with each other. After exploring and cleaning our data a little, we're going to answer questions like:</p>
<ul>
<li>What are the most extreme game outcomes?</li>
<li>How does the game affect television viewership?</li>
<li>How have viewership, TV ratings, and ad cost evolved over time?</li>
<li>Who are the most prolific musicians in terms of halftime show performances?</li>
</ul>
<p><img src="https://assets.datacamp.com/production/project_731/img/left_shark.jpg" alt="Left Shark Steals The Show">
<em><a href="https://www.flickr.com/photos/huntleypaton/16464994135/in/photostream/">Left Shark Steals The Show</a>. Katy Perry performing at halftime of Super Bowl XLIX. Photo by Huntley Paton. Attribution-ShareAlike 2.0 Generic (CC BY-SA 2.0).</em></p>
<p>The database we'll use is named <code>super_bowls</code>. It was <a href="https://en.wikipedia.org/wiki/Web_scraping">scraped</a> and polished from Wikipedia and is made up of three tables, one with <a href="https://en.wikipedia.org/wiki/List_of_Super_Bowl_champions">game data</a>, one with <a href="https://en.wikipedia.org/wiki/Super_Bowl_television_ratings">TV broadcast data</a>, and one with <a href="https://en.wikipedia.org/wiki/List_of_Super_Bowl_halftime_shows">halftime musician performance data</a> for all 52 Super Bowls through 2018. Let's take a look at the game data, which is stored in the <code>games</code> table.</p>

In [156]:
%%sql
postgresql:///super_bowls
    
select * 
from games
order by super_bowl desc
limit 5;



5 rows affected.


[(52, datetime.date(2018, 2, 4), 'U.S. Bank Stadium', 'Minneapolis', 'Minnesota', 67612, 'Philadelphia Eagles', 41, 'Nick Foles', None, 'Doug Pederson', 'New England Patriots', 33, 'Tom Brady', None, 'Bill Belichick', 74, 8),
 (51, datetime.date(2017, 2, 5), 'NRG Stadium', 'Houston', 'Texas', 70807, 'New England Patriots', 34, 'Tom Brady', None, 'Bill Belichick', 'Atlanta Falcons', 28, 'Matt Ryan', None, 'Dan Quinn', 62, 6),
 (50, datetime.date(2016, 2, 7), "Levi's Stadium", 'Santa Clara', 'California', 71088, 'Denver Broncos', 24, 'Peyton Manning', None, 'Gary Kubiak', 'Carolina Panthers', 10, 'Cam Newton', None, 'Ron Rivera', 34, 14),
 (49, datetime.date(2015, 2, 1), 'University of Phoenix Stadium', 'Glendale', 'Arizona', 70288, 'New England Patriots', 28, 'Tom Brady', None, 'Bill Belichick', 'Seattle Seahawks', 24, 'Russell Wilson', None, 'Pete Carroll', 52, 4),
 (48, datetime.date(2014, 2, 2), 'MetLife Stadium', 'East Rutherford', 'New Jersey', 82529, 'Seattle Seahawks', 43, 'Russell Wilson', None, 'Pete Carroll', 'Denver Broncos', 8, 'Peyton Manning', None, 'John Fox', 51, 35)]

## 2. Data issues in the broadcasts table
<p>For the Super Bowl game data, we can see the dataset appears whole except for missing values in the backup quarterback columns (<code>qb_winner_2</code> and <code>qb_loser_2</code>), which make sense given most starting QBs in the Super Bowl (<code>qb_winner_1</code> and <code>qb_loser_1</code>) play the entire game.</p>
<p>Let's take a look at the TV broadcast data, which is stored in the <code>broadcasts</code> table.</p>

In [158]:
%%sql

select * 
from broadcasts
limit 10


 * postgresql:///super_bowls
10 rows affected.


[(1, 1, 'NBC', 24430000, None, 18.5, 36, None, None, 37500),
 (2, 1, 'CBS', 26750000, 51180000, 22.6, 43, None, None, 42500),
 (3, 2, 'CBS', 39120000, None, 36.8, 68, None, None, 54500),
 (4, 3, 'NBC', 41660000, None, 36.0, 70, None, None, 55000),
 (5, 4, 'CBS', 44270000, None, 39.4, 69, None, None, 78200),
 (6, 5, 'NBC', 46040000, None, 39.9, 75, None, None, 72500),
 (7, 6, 'CBS', 56640000, None, 44.2, 74, None, None, 86100),
 (8, 7, 'NBC', 53320000, None, 42.7, 72, None, None, 88100),
 (9, 8, 'CBS', 51700000, None, 41.6, 73, None, None, 103500),
 (10, 9, 'NBC', 56050000, None, 42.4, 72, None, None, 107000)]

## 3. Data issues in the performances table
<p>For the TV data, the following columns have missing values and a lot of them:</p>
<ul>
<li><code>total_us_viewers</code> (amount of U.S. viewers who watched at least some part of the broadcast)</li>
<li><code>rating_18_49</code> (average % of U.S. adults 18-49 who live in a household with a TV that were watching for the entire broadcast)</li>
<li><code>share_18_49</code> (average % of U.S. adults 18-49 who live in a household with a TV <em>in use</em> that were watching for the entire broadcast)</li>
</ul>
<p>Let's check out the halftime musician performance data (stored in the <code>performances</code> table). We'll look at one </p>

In [160]:
%%sql

select * from performances 
where num_songs is NULL
order by super_bowl desc


 * postgresql:///super_bowls
46 rows affected.


[(121, 49, 'Arizona State University Sun Devil Marching Band', None),
 (100, 41, 'Florida A&M University Marching 100 Band', None),
 (91, 38, 'The Ocean of Soul Marching Band', None),
 (90, 38, 'The Spirit of Houston Cougar Marching Band', None),
 (53, 26, 'University of Minnesota Marching Band', None),
 (46, 24, 'The Human Jukebox', None),
 (47, 24, 'Pride of Nicholls Marching Band', None),
 (45, 24, 'Pride of Acadiana', None),
 (42, 22, 'San Diego State University Marching Aztecs', None),
 (41, 22, 'Spirit of Troy', None),
 (38, 20, 'Up with People', None),
 (37, 19, 'Tops In Blue', None),
 (34, 17, 'Los Angeles Unified School District All City Honor Marching Band', None),
 (33, 16, 'Up with People', None),
 (32, 15, 'The Human Jukebox', None),
 (31, 15, "Helen O'Connell", None),
 (30, 14, 'Up with People', None),
 (29, 14, 'Grambling State University Tiger Marching Band', None),
 (28, 13, 'Ken Hamilton', None),
 (27, 13, 'Gramacks', None),
 (24, 12, 'Al Hirt', None),
 (25, 12, 'Pete Fountain', None),
 (26, 12, 'Tyler Junior College Apache Band', None),
 (23, 11, 'Los Angeles Unified School District All City Honor Marching Band', None),
 (22, 10, 'Up with People', None),
 (21, 9, 'Grambling State University Tiger Marching Band', None),
 (20, 9, 'Mercer Ellington', None),
 (18, 8, 'University of Texas Longhorn Band', None),
 (19, 8, 'Judy Mallett', None),
 (16, 7, 'Woody Herman', None),
 (17, 7, 'University of Michigan Marching Band', None),
 (15, 7, 'Andy Williams', None),
 (13, 6, 'Carol Channing', None),
 (14, 6, 'Ella Fitzgerald', None),
 (12, 6, 'Al Hirt', None),
 (11, 6, 'United States Air Force Academy Cadet Chorale', None),
 (10, 5, 'Southeast Missouri State Marching Band', None),
 (9, 4, 'Marguerite Piazza', None),
 (8, 4, 'Doc Severinsen', None),
 (7, 4, 'Al Hirt', None),
 (6, 4, 'The Human Jukebox', None),
 (5, 3, 'Florida A&M University Marching 100 Band', None),
 (4, 2, 'Grambling State University Tiger Marching Band', None),
 (2, 1, 'Grambling State University Tiger Marching Band', None),
 (1, 1, 'Al Hirt', None),
 (3, 1, 'University of Arizona Symphonic Marching Band', None)]

## 4. The highest- and lowest-scoring Super Bowls
<p>We have missing numbers of songs performed (<code>num_songs</code>) for about a third of the performances, most of them for earlier Super Bowls.</p>
<p>Why all of these missing values in the <code>broadcasts</code> and <code>performances</code> tables? Was the data ever tracked? Was it lost in history? Is this a Wikipedia issue? There are a lot of potential reasons. Is the research effort to make this data whole worth it? Maybe. Watching every Super Bowl halftime show to get song counts would be pretty fun. But we don't have the time to do that kind of stuff now! Let's take note of where the dataset isn't perfect and start uncovering some insights.</p>
<p>Let's start by looking at combined points for each Super Bowl by pinpointing the Super Bowls with the highest and lowest scores.</p>

In [162]:
%%sql

select * 
from games
where combined_pts < 25 OR combined_pts > 70


 * postgresql:///super_bowls
5 rows affected.


[(3, datetime.date(1969, 1, 12), 'Orange Bowl', 'Miami', 'Florida', 75389, 'New York Jets', 16, 'Joe Namath', None, 'Weeb Ewbank', 'Baltimore Colts', 7, 'Earl Morrall', 'Johnny Unitas', 'Don Shula', 23, 9),
 (7, datetime.date(1973, 1, 14), 'Memorial Coliseum', 'Los Angeles', 'California', 90182, 'Miami Dolphins', 14, 'Bob Griese', None, 'Don Shula', 'Washington Redskins', 7, 'Bill Kilmer', None, 'George Allen', 21, 7),
 (9, datetime.date(1975, 1, 12), 'Tulane Stadium', 'New Orleans', 'Louisiana', 80997, 'Pittsburgh Steelers', 16, 'Terry Bradshaw', None, 'Chuck Noll', 'Minnesota Vikings', 6, 'Fran Tarkenton', None, 'Bud Grant', 22, 10),
 (29, datetime.date(1995, 1, 29), 'Joe Robbie Stadium', 'Miami Gardens', 'Florida', 74107, 'San Francisco 49ers', 49, 'Steve Young', None, 'George Seifert', 'San Diego Chargers', 26, 'Stan Humphreys', None, 'Bobby Ross', 75, 23),
 (52, datetime.date(2018, 2, 4), 'U.S. Bank Stadium', 'Minneapolis', 'Minnesota', 67612, 'Philadelphia Eagles', 41, 'Nick Foles', None, 'Doug Pederson', 'New England Patriots', 33, 'Tom Brady', None, 'Bill Belichick', 74, 8)]

## 5. The blowouts and nailbiters
<p>The highest combined scores at 74 and 75, we find two games featuring dominant quarterback performances. One even happened recently in 2018's Super Bowl LII where Tom Brady's Patriots lost to Nick Foles' underdog Eagles 41-33 for a combined score of 74.</p>
<p>Going down to the lowest combined scores, we have Super Bowl III and VII, which featured tough defenses that dominated. We also have Super Bowl IX in New Orleans in 1975, whose 16-6 score can be attributed to inclement weather. The field was slick from overnight rain, and it was cold at 46 °F (8 °C), making it hard for the Steelers and Vikings to do much offensively. This was the second-coldest Super Bowl ever and the last to be played in inclement weather for over 30 years. The NFL realized people like points, I guess.</p>
<p><em>UPDATE: In Super Bowl LIII in 2019, the Patriots and Rams broke the record for the lowest-scoring Super Bowl with a combined score of 16 points (13-3 for the Patriots).</em></p>
<p>Let's take a look at point <em>difference</em> now.</p>

In [164]:
%%sql

select * 
from games
where difference_pts = 1 OR difference_pts >= 35


 * postgresql:///super_bowls
5 rows affected.


[(20, datetime.date(1986, 1, 26), 'Louisiana Superdome', 'New Orleans', 'Louisiana', 73818, 'Chicago Bears', 46, 'Jim McMahon', None, 'Mike Ditka', 'New England Patriots', 10, 'Tony Eason', 'Steve Grogan', 'Raymond Berry', 56, 36),
 (24, datetime.date(1990, 1, 28), 'Louisiana Superdome', 'New Orleans', 'Louisiana', 72919, 'San Francisco 49ers', 55, 'Joe Montana', None, 'George Seifert', 'Denver Broncos', 10, 'John Elway', None, 'Dan Reeves', 65, 45),
 (25, datetime.date(1991, 1, 27), 'Tampa Stadium', 'Tampa', 'Florida', 73813, 'New York Giants', 20, 'Jeff Hostetler', None, 'Bill Parcells', 'Buffalo Bills', 19, 'Jim Kelly', None, 'Marv Levy', 39, 1),
 (27, datetime.date(1993, 1, 31), 'Rose Bowl', 'Pasadena', 'California', 98374, 'Dallas Cowboys', 52, 'Troy Aikman', None, 'Jimmy Johnson', 'Buffalo Bills', 17, 'Jim Kelly', 'Frank Reich', 'Marv Levy', 69, 35),
 (48, datetime.date(2014, 2, 2), 'MetLife Stadium', 'East Rutherford', 'New Jersey', 82529, 'Seattle Seahawks', 43, 'Russell Wilson', None, 'Pete Carroll', 'Denver Broncos', 8, 'Peyton Manning', None, 'John Fox', 51, 35)]

## 6. Do blowouts translate to lost viewers?
<p>The closest game ever was when the Buffalo Bills lost to the New York Giants by 1 point in 1991, which was  best remembered for Scott Norwood's last-second missed field goal attempt that went <em><a href="https://www.youtube.com/watch?v=RPFZCGgjDSg">wide right</a></em>, kicking off four Bills Super Bowl losses in a row. Poor Scott. The biggest point discrepancy ever was 45 points (!) where Hall of Famer Joe Montana's led the San Francisco 49ers to victory in 1990, one year before the closest game ever.</p>
<p>I remember watching the Seahawks crush the Broncos by 35 points (43-8) in 2014, which was a boring experience in my opinion. The game was never really close. I'm pretty sure we changed the channel at the end of the third quarter. Let's combine our game data and TV data to see if this is a universal phenomenon. Do large point differences translate to lost viewers? Let's compare <a href="https://en.wikipedia.org/wiki/Nielsen_ratings">household share</a> <em>(average percentage of U.S. households with a TV in use that were watching for the entire broadcast)</em> for "blowouts" (games where the point difference is three touchdowns, i.e., 21 points,  or more) vs. non-blowouts.</p>

In [166]:
%%sql
SELECT blowout,
       AVG(difference_pts) as avg_difference_pts,
       AVG(share_household) as avg_share_household
FROM
    (SELECT games.super_bowl, difference_pts, share_household,
        CASE WHEN difference_pts >= 21 THEN TRUE
        ELSE FALSE END
        AS blowout
    FROM games
    INNER JOIN broadcasts
    ON games.super_bowl = broadcasts.super_bowl) AS subquery
group by blowout;

 * postgresql:///super_bowls
2 rows affected.


[(False, Decimal('9.3250000000000000'), Decimal('67.8250000000000000')),
 (True, Decimal('29.3846153846153846'), Decimal('61.9230769230769231'))]

## 7. Viewership and the ad industry over time
<p>We see a 6-point difference in average household share for blowouts vs. non-blowouts! Though it matches our intuition, we must take it with a grain of salt due to our small sample size of 52 games.</p>
<p>Regardless of the score though, I bet most people stick it out for the halftime show, which is good news for the TV networks and advertisers. A 30-second spot costs a pretty <a href="https://www.businessinsider.com/super-bowl-commercials-cost-more-than-eagles-quarterback-earns-2018-1">\$5 million</a> now, but has it always been that way? And how have number of viewers and household ratings trended alongside ad cost? Let's take a look at this data in six different decades.</p>

In [168]:
%%sql

select super_bowl, avg_us_viewers, rating_household, ad_cost
from broadcasts
where super_bowl in (2, 12, 22, 32, 42, 52)

 * postgresql:///super_bowls
6 rows affected.


[(2, 39120000, 36.8, 54500),
 (12, 78940000, 47.2, 162300),
 (22, 80140000, 41.9, 645000),
 (32, 90000000, 44.5, 1291100),
 (42, 97448000, 43.1, 2699963),
 (52, 103390000, 43.1, 5000000)]

## 8. Halftime shows weren't always this great
<p>We can see viewers increased before ad costs did. <code>avg_us_viewers</code> was 90 million in 1998 (Super Bowl XXXII) and an ad cost \$1.29 million. That is an <a href="https://westegg.com/inflation/">inflation-adjusted</a> \$2.02 million in 2018, where <code>avg_us_viewers</code> was actually 103 million and <code>ad_cost</code> was \$5 million. Maybe the networks weren't very data savvy and were slow to react. (It'd make sense since DataCamp didn't exist back then!)</p>
<p>Another hypothesis: maybe halftime shows weren't that good in the earlier years? The modern spectacle of the Super Bowl has a lot to do with the cultural prestige of big halftime acts. I went down a YouTube rabbit hole and it turns out the old ones weren't up to today's standards. Some offenders:</p>
<ul>
<li><a href="https://youtu.be/6wMXHxWO4ns?t=263">Super Bowl XXVI</a> in 1992: A Frosty The Snowman rap performed by children.</li>
<li><a href="https://www.youtube.com/watch?v=PKQTL1PYSag">Super Bowl XXIII</a> in 1989: An Elvis impersonator that did magic tricks and didn't even sing one Elvis song.</li>
<li><a href="https://youtu.be/oSXMNbK2e98?t=436">Super Bowl XXI</a> in 1987: Tap dancing ponies. (Okay, that's pretty awesome actually.)</li>
</ul>
<p>It turns out Michael Jackson's Super Bowl XXVII performance, one of the most watched events in American TV history, was when the NFL realized the value of Super Bowl airtime and decided they needed to sign big name acts from then on out. The halftime shows before MJ indeed weren't that impressive, which we can see by filtering our <code>performances</code> table.</p>

In [170]:
%%sql
SELECT *
FROM performances
WHERE super_bowl <= 
    (SELECT super_bowl
     FROM performances
     WHERE musician like 'Michael Jackson')
ORDER BY super_bowl DESC;

 * postgresql:///super_bowls
54 rows affected.


[(54, 27, 'Michael Jackson', 5),
 (53, 26, 'University of Minnesota Marching Band', None),
 (52, 26, 'Gloria Estefan', 2),
 (51, 25, 'New Kids on the Block', 2),
 (47, 24, 'Pride of Nicholls Marching Band', None),
 (46, 24, 'The Human Jukebox', None),
 (45, 24, 'Pride of Acadiana', None),
 (50, 24, 'Pete Fountain', 1),
 (49, 24, 'Doug Kershaw', 1),
 (48, 24, 'Irma Thomas', 1),
 (44, 23, 'Elvis Presto', 7),
 (42, 22, 'San Diego State University Marching Aztecs', None),
 (43, 22, 'Chubby Checker', 2),
 (41, 22, 'Spirit of Troy', None),
 (40, 21, 'Grambling State University Tiger Marching Band', 8),
 (39, 21, 'Spirit of Troy', 8),
 (38, 20, 'Up with People', None),
 (37, 19, 'Tops In Blue', None),
 (35, 18, "The University of Florida Fightin' Gator Marching Band", 7),
 (36, 18, 'The Florida State University Marching Chiefs', 7),
 (34, 17, 'Los Angeles Unified School District All City Honor Marching Band', None),
 (33, 16, 'Up with People', None),
 (32, 15, 'The Human Jukebox', None),
 (31, 15, "Helen O'Connell", None),
 (29, 14, 'Grambling State University Tiger Marching Band', None),
 (30, 14, 'Up with People', None),
 (28, 13, 'Ken Hamilton', None),
 (27, 13, 'Gramacks', None),
 (24, 12, 'Al Hirt', None),
 (26, 12, 'Tyler Junior College Apache Band', None),
 (25, 12, 'Pete Fountain', None),
 (23, 11, 'Los Angeles Unified School District All City Honor Marching Band', None),
 (22, 10, 'Up with People', None),
 (20, 9, 'Mercer Ellington', None),
 (21, 9, 'Grambling State University Tiger Marching Band', None),
 (19, 8, 'Judy Mallett', None),
 (18, 8, 'University of Texas Longhorn Band', None),
 (16, 7, 'Woody Herman', None),
 (17, 7, 'University of Michigan Marching Band', None),
 (15, 7, 'Andy Williams', None),
 (11, 6, 'United States Air Force Academy Cadet Chorale', None),
 (14, 6, 'Ella Fitzgerald', None),
 (13, 6, 'Carol Channing', None),
 (12, 6, 'Al Hirt', None),
 (10, 5, 'Southeast Missouri State Marching Band', None),
 (7, 4, 'Al Hirt', None),
 (9, 4, 'Marguerite Piazza', None),
 (8, 4, 'Doc Severinsen', None),
 (6, 4, 'The Human Jukebox', None),
 (5, 3, 'Florida A&M University Marching 100 Band', None),
 (4, 2, 'Grambling State University Tiger Marching Band', None),
 (2, 1, 'Grambling State University Tiger Marching Band', None),
 (3, 1, 'University of Arizona Symphonic Marching Band', None),
 (1, 1, 'Al Hirt', None)]

## 9. Who has the most halftime show appearances?
<p>Lots of marching bands. American jazz clarinetist Pete Fountain. Miss Texas 1973 playing a violin. Nothing against those performers, they're just simply not <a href="https://www.youtube.com/watch?v=suIg9kTGBVI">Beyoncé</a>. To be fair, no one is.</p>
<p>Let's see all of the musicians that have done at least one halftime show, including their performance counts.</p>

In [172]:
%%sql
SELECT musician, count(*) as count_performances
FROM performances
GROUP BY  musician
HAVING COUNT(*) >1
ORDER BY count_performances DESC, musician ASC;

 * postgresql:///super_bowls
14 rows affected.


[('Grambling State University Tiger Marching Band', 6),
 ('Al Hirt', 4),
 ('Up with People', 4),
 ('The Human Jukebox', 3),
 ('Beyoncé', 2),
 ('Bruno Mars', 2),
 ('Florida A&M University Marching 100 Band', 2),
 ('Gloria Estefan', 2),
 ('Justin Timberlake', 2),
 ('Los Angeles Unified School District All City Honor Marching Band', 2),
 ('Nelly', 2),
 ('Pete Fountain', 2),
 ('Spirit of Troy', 2),
 ('University of Minnesota Marching Band', 2)]

## 10. Who performed the most songs in a halftime show?
<p>The world famous <a href="https://www.youtube.com/watch?v=RL_3oqpHiDg">Grambling State University Tiger Marching Band</a> takes the crown with six appearances. Beyoncé, Justin Timberlake, Nelly, and Bruno Mars are the only post-Y2K musicians with multiple appearances (two each).</p>
<p>From our previous inspections, the <code>num_songs</code> column has lots of missing values:</p>
<ul>
<li>A lot of the marching bands don't have <code>num_songs</code> entries.</li>
<li>For non-marching bands, missing data starts occurring at Super Bowl XX.</li>
</ul>
<p>We can filter out <em>most</em> marching bands by filtering out musicians with the word "Marching" and the word "Spirit" in them (a common naming convention for marching bands is "Spirit of [something]"). Then we'll filter for Super Bowls after Super Bowl XX to address the missing data issue, <em>then</em> let's see who has the most number of songs.</p>

In [174]:
%%sql
select *
from performances
where musician NOT LIKE '%Marching%' AND
musician NOT LIKE '%Spirit%' AND
num_songs is NOT NULL
ORDER BY num_songs DESC


 * postgresql:///super_bowls
81 rows affected.


[(134, 52, 'Justin Timberlake', 11),
 (64, 30, 'Diana Ross', 10),
 (124, 49, 'Katy Perry', 8),
 (118, 47, 'Beyoncé', 7),
 (44, 23, 'Elvis Presto', 7),
 (101, 41, 'Prince', 7),
 (132, 51, 'Lady Gaga', 7),
 (131, 50, 'Coldplay', 6),
 (120, 48, 'Bruno Mars', 6),
 (109, 45, 'The Black Eyed Peas', 6),
 (54, 27, 'Michael Jackson', 5),
 (104, 44, 'The Who', 5),
 (114, 46, 'Madonna', 5),
 (75, 33, 'Stevie Wonder', 4),
 (69, 32, 'The Temptations', 4),
 (98, 39, 'Paul McCartney', 4),
 (103, 43, 'Bruce Springsteen and the E Street Band', 4),
 (102, 42, 'Tom Petty & the Heartbreakers', 4),
 (65, 31, 'James Brown', 3),
 (85, 35, 'Aerosmith', 3),
 (86, 36, 'U2', 3),
 (129, 50, 'Bruno Mars', 3),
 (95, 38, 'P. Diddy', 3),
 (96, 38, 'Janet Jackson', 3),
 (99, 40, 'The Rolling Stones', 3),
 (63, 29, 'Patti Labelle', 3),
 (84, 35, 'NSYNC', 3),
 (66, 31, 'ZZ Top', 3),
 (67, 31, 'The Blues Brothers', 3),
 (70, 32, 'Boyz II Men', 3),
 (130, 50, 'Beyoncé', 3),
 (73, 32, 'Smokey Robinson', 3),
 (126, 50, 'Youth Orchestra Los Angeles', 3),
 (76, 33, 'Gloria Estefan', 3),
 (125, 50, 'Gustavo Dudamel', 3),
 (122, 49, 'Missy Elliott', 3),
 (88, 37, 'Shania Twain', 2),
 (51, 25, 'New Kids on the Block', 2),
 (52, 26, 'Gloria Estefan', 2),
 (55, 28, 'Wynonna Judd', 2),
 (57, 28, 'Travis Tritt', 2),
 (58, 28, 'Tanya Tucker', 2),
 (59, 28, 'Clint Black', 2),
 (61, 29, 'Arturo Sandoval', 2),
 (62, 29, 'Tony Bennett', 2),
 (71, 32, 'Queen Latifah', 2),
 (72, 32, 'Martha Reeves', 2),
 (87, 37, 'No Doubt', 2),
 (43, 22, 'Chubby Checker', 2),
 (93, 38, 'Kid Rock', 2),
 (110, 46, 'Cee Lo Green', 2),
 (117, 47, "Destiny's Child", 2),
 (92, 38, 'Justin Timberlake', 1),
 (89, 37, 'Sting', 1),
 (105, 45, 'Fergie', 1),
 (106, 45, 'will.i.am', 1),
 (107, 45, 'Usher', 1),
 (108, 45, 'Slash', 1),
 (83, 35, 'Britney Spears', 1),
 (49, 24, 'Doug Kershaw', 1),
 (111, 46, 'M.I.A.', 1),
 (112, 46, 'Nicki Minaj', 1),
 (113, 46, 'LMFAO', 1),
 (82, 35, 'Mary J. Blige', 1),
 (115, 47, 'Michelle Williams', 1),
 (116, 47, 'Kelly Rowland', 1),
 (48, 24, 'Irma Thomas', 1),
 (81, 35, 'Nelly', 1),
 (119, 48, 'Red Hot Chili Peppers', 1),
 (80, 34, 'Phil Collins', 1),
 (79, 34, 'Christina Aguilera', 1),
 (123, 49, 'Lenny Kravitz', 1),
 (78, 34, 'Enrique Iglesias', 1),
 (77, 34, 'Toni Braxton', 1),
 (74, 33, 'Big Bad Voodoo Daddy', 1),
 (128, 50, 'Mark Ronson', 1),
 (60, 29, 'Miami Sound Machine', 1),
 (56, 28, 'The Judds', 1),
 (50, 24, 'Pete Fountain', 1),
 (97, 38, 'Jessica Simpson', 1),
 (94, 38, 'Nelly', 1)]

## 11. Conclusion
<p>So most non-band musicians do 1-3 songs per halftime show. It's important to note that the duration of the halftime show is fixed (roughly 12 minutes) so songs per performance is more a measure of how many hit songs you have. JT went off in 2018, wow. 11 songs! Diana Ross comes in second with 10 in her medley in 1996.</p>
<p>In this notebook, we loaded, cleaned, then explored Super Bowl game, television, and halftime show data. We visualized the distributions of combined points, point differences, and halftime show performances using histograms. We used line plots to see how ad cost increases lagged behind viewership increases. And we discovered that blowouts do appear to lead to a drop in viewers.</p>
<p>This year's Big Game features the Rams and the Patriots, and these two teams have actually met before in a previous Super Bowl. Who will win this time? Will the score be similar?</p>
<p><em>UPDATE: <a href="https://en.wikipedia.org/wiki/Super_Bowl_LIII">Spoiler alert</a>.</em></p>

In [176]:
%%sql

select *
from
games
where team_winner like '%Patriots%' AND
team_loser like '%Rams%'


 * postgresql:///super_bowls
1 rows affected.


[(36, datetime.date(2002, 2, 3), 'Louisiana Superdome', 'New Orleans', 'Louisiana', 72922, 'New England Patriots', 20, 'Tom Brady', None, 'Bill Belichick', 'St. Louis Rams', 17, 'Kurt Warner', None, 'Mike Martz', 37, 3)]